<a href="https://colab.research.google.com/github/DeveloperPratim/sses_final/blob/main/Subjective_Student_Evolution_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For Hugging Face Transformers
!pip install transformers

# For Sentence Transformers
!pip install sentence-transformers

# For BERTScore
!pip install bert-score

# For Scipy (includes euclidean and cityblock)
!pip install scipy

# For Scikit-learn (includes cosine_similarity)
!pip install scikit-learn

# For NumPy (used for standard deviation)
!pip install numpy

MyModel For Marks Calculation

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Define your Google Drive path
drive_path = "/content/drive/MyDrive/MyModel"
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=drive_path)
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=drive_path)

# Initialize the pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Model For Simentic Similarity Calculation

In [ ]:
from sentence_transformers import SentenceTransformer


# Initialize models if not already initialized
try:
    # Check for SentenceTransformer model
    if 'sentence_transformer_model' not in globals():
        sentence_transformer_model = SentenceTransformer("all-MiniLM-L6-v2")
        print("SentenceTransformer model initialized.")
    else:
        print("SentenceTransformer model already initialized.")

except Exception as e:
    print(f"Error during model initialization: {e}")

Defining All Functions Required

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean, cityblock
from scipy.stats import pearsonr
from sentence_transformers import SentenceTransformer
import numpy as np
import bert_score
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


def extract_score_feedback_and_similarity(generated_text, semantic_similarity):
    # Regular expression patterns
    score_pattern = r'"score":\s*([0-9.]+)'
    feedback_pattern = r'"feedback":\s*"([^"]+)"'

    # Find matches
    score_match = re.search(score_pattern, generated_text)
    feedback_match = re.search(feedback_pattern, generated_text)

    # Extract values
    score = float(score_match.group(1)) if score_match else None
    feedback = feedback_match.group(1) if feedback_match else None

    # Return the result with semantic_similarity
    return {
        "score": score,
        "feedback": feedback,
        "semantic_similarity": semantic_similarity
    }


def compute_similarity(correct_answer, student_answer, model_name="all-MiniLM-L6-v2"):
    # Encode the answers into embeddings (vector representations)
    embeddings = sentence_transformer_model.encode([correct_answer, student_answer])

    # Cosine similarity
    cosine_sim = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

    # Euclidean distance
    euclidean_dist = euclidean(embeddings[0], embeddings[1])

    # Manhattan distance
    manhattan_dist = cityblock(embeddings[0], embeddings[1])

    # Pearson correlation
    pearson_corr, _ = pearsonr(embeddings[0], embeddings[1])

    # Jaccard Similarity
    set_correct = set(correct_answer.split())
    set_student = set(student_answer.split())
    jaccard_sim = len(set_correct.intersection(set_student)) / len(set_correct.union(set_student))

    # BERTScore
    P, R, F1 = bert_score.score([correct_answer], [student_answer], lang="en")
    bert_score_sim = F1.mean().item()

    return {
        "cosine_similarity": cosine_sim,
        "euclidean_distance": euclidean_dist,
        "manhattan_distance": manhattan_dist,
        "pearson_correlation": pearson_corr,
        "jaccard_similarity": jaccard_sim,
        "bert_score_similarity": bert_score_sim
    }

import numpy as np

def compute_final_marks(similarities, max_marks, score, feedback, question, student_answer, actual_answer):
    # Apply adjustments to score based on the provided conditions
    if score < 1.5:
        score -= 0.92
    elif score < 3.5:
        score -= 0.56
    elif score < 4.8:
        score -= 0.12

    # Normalize the score after adjustments
    adjusted_score = max(0, min(score, max_marks))  # Ensure score is within 0 to max_marks

    # Standard deviation of the similarity scores (using the list of similarity scores)
    similarity_values = [
        similarities['cosine_similarity'],
        similarities['pearson_correlation'],
        similarities['bert_score_similarity'],
        similarities['jaccard_similarity'],
        similarities['euclidean_distance'],
        similarities['manhattan_distance']
    ]

    # Calculate the standard deviation (SD) of the similarity scores
    similarity_sd = np.std(similarity_values)

    # Scale or limit similarity_sd adjustment
    similarity_sd = min(similarity_sd, 0.5 * max_marks)  # Limit the impact of SD

    # Adjust the final score based on similarity SD
    final_score = adjusted_score - similarity_sd

    # Ensure the final score does not exceed max_marks
    final_marks = max(0, min(final_score, max_marks))

    # Calculate percentage (scaled to max_marks)
    percentage = (final_marks / max_marks) * 100

    # Return the results with necessary fields
    return {
        'final_marks': final_marks,
        'max_marks': max_marks,
        'marks_obtained': adjusted_score,
        'feedback': feedback,
        'question': question,
        'student_answer': student_answer,
        'actual_answer': actual_answer,
        'similarity_sd': similarity_sd,
        'percentage': percentage
    }


def get_data(question, student_answer, actual_answer, max_marks):
    return {
        'question': question,
        'student_answer': student_answer,
        'actual_answer': actual_answer,
        'max_marks': max_marks
    }


def evaluate_student_answer(pipe, question, correct_answer, student_answer, max_marks):
    # Dynamically set max token limit
    input_token_count = len(pipe.tokenizer.encode(f"Question: {question} Answer: {student_answer}"))
    max_allowed_tokens = 1024  # Adjust based on your model's total token limit
    max_new_tokens = max(50, max_allowed_tokens - input_token_count)  # Ensure a minimum response length

    input_text = f"""
    Question: {question}
    Answer: {student_answer}
    Evaluate and provide these two fields **score** and **feedback** based on this answer where full marks is 5 and can be given for most precious and best answer and 0 for wrong or irrelevant answer and in response just give two fields in json, with no extra text.
    """

    # Generate output with dynamic token limit from model
    result = pipe(input_text, max_new_tokens=max_new_tokens, num_return_sequences=1, truncation=True)
    generated_text = result[0]['generated_text']

    # Extract score and feedback from model output
    semantic_similarity = compute_similarity(correct_answer, student_answer)["cosine_similarity"]
    score_feedback = extract_score_feedback_and_similarity(generated_text, semantic_similarity)

    global score
    # Use the score and feedback from model
    score = score_feedback["score"]
    feedback = score_feedback["feedback"]

    # Collect similarities for final marks computation
    similarities = compute_similarity(correct_answer, student_answer)

    # Get the required data for final marks calculation
    data = get_data(question, student_answer, correct_answer, max_marks)

    # Compute final marks and return all values
    final_result = compute_final_marks(similarities, max_marks, score, feedback, data['question'], data['student_answer'], data['actual_answer'])

    # Print the final result in a more readable format
    print(f"Question: {final_result['question']}")
    print(f"Student's Answer: {final_result['student_answer']}")
    print(f"Correct Answer: {final_result['actual_answer']}")
    print(f"Maximum Marks: {final_result['max_marks']}")
    print(f"Marks Obtained: {final_result['marks_obtained']}")
    print(f"Final Marks: {final_result['final_marks']}")
    print(f"Feedback: {final_result['feedback']}")
    print(f"Percentage: {final_result['marks_obtained'] / final_result['max_marks'] * 100:.2f}%")
    print(f"Similarity Metrics:")
    print(f"   Cosine Similarity: {similarities['cosine_similarity']:.2f}")
    print(f"   Pearson Correlation: {similarities['pearson_correlation']:.2f}")
    print(f"   BERTScore Similarity: {similarities['bert_score_similarity']:.2f}")
    print(f"   Jaccard Similarity: {similarities['jaccard_similarity']:.2f}")
    print(f"   Euclidean Distance: {similarities['euclidean_distance']:.2f}")
    print(f"   Manhattan Distance: {similarities['manhattan_distance']:.2f}")

    return final_result

Using Those Function to See the Output

In [ ]:

# Example usage
question = "What is the role of an operating system?"
correct_answer = "The operating system manages the hardware and software resources of a computer."
student_answer = """

operating system (OS) manages a computer's hardware and software, and acts as a bridge between the user and the computer. Some of its key roles include:
Memory management
Allocates and tracks memory for applications and processes, ensuring they have enough to run and that the system performs well. This is especially important for computers with limited memory.
Process management
Controls how programs interact with each other, and ensures that resources are used efficiently and the system is stable.
File system management
Organizes and manages files on storage devices, allowing users to create, read, write, and delete files.
User interface
Provides a user-friendly interface for interacting with the computer, either through a graphical or command-line interface.
Other functions of an operating system include: Processor management and scheduling, Device management, and Storage management.


"""  # The student's answer
max_marks = 5  # The maximum marks for the question

# Assume `pipe` is your language model pipeline
final_result = evaluate_student_answer(pipe, question, correct_answer, student_answer, max_marks)

In [ ]:
print(f"Score: {score}")